<a href="https://colab.research.google.com/github/zabir-nabil/medical-image-TL/blob/master/X_Ray_TB_Segmentation_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('zabiralnazi@yahoo.com')

zabiralnazi@yahoo.com


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
% cd /content/drive/My Drive/x_ray_tb/

/content/drive/My Drive/x_ray_tb


In [0]:
import numpy as np

imgs = np.load('imgs.npy')

In [5]:
imgs.shape

(138, 224, 224, 3)

In [6]:
masks = np.load('masks.npy')
masks.shape

(138, 224, 224)

In [7]:
y = np.load('y.npy')
y.shape

(138, 2)

In [0]:
! pip install Augmentor 

In [9]:
print(imgs.shape)
print(masks.shape)

(138, 224, 224, 3)
(138, 224, 224)


In [10]:
masks = np.reshape(masks, (138,224,224,1))
print(masks.shape)

(138, 224, 224, 1)


In [0]:
z = []
for i in range(138):
  z.append([imgs[i,:,:,:], masks[i,:,:,:]])

In [12]:
print(len(z))
print(len(z[0]))
print(z[0][0].shape)
print(z[0][1].shape)

138
2
(224, 224, 3)
(224, 224, 1)


In [16]:
import keras.backend as K

Using TensorFlow backend.


In [0]:
def dice_coef(y_true, y_pred):
    y_true_f = (y_true).flatten()
    y_pred_f = (y_pred).flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    # +1 added to avoid 0/0 division
    return (2.0 * intersection + 1.0) / (np.sum(y_true_f) + np.sum(y_pred_f) + 1.0)
  
def jacard_coef(y_true, y_pred):
    y_true_f = (y_true).flatten()
    y_pred_f = (y_pred).flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (np.sum(y_true_f) + np.sum(y_pred_f) - intersection + 1.0)
  
  
def weighted_loss(y_true, y_pred):
  loss1 = dice_coef(y_true, y_pred)
  loss2 = jacard_coef(y_true, y_pred)
  return -(loss1*0.4 + loss2*0.6)

In [0]:
ax[ax>0.5] = 1.0
ax[ax<=0.5] = 0.0

In [38]:
#ax = np.random.rand(12,12)
#ay = np.random.rand(12,12)

print(jacard_coef(ax, ax))
print(jacard_coef(ay, ay))

1.0
0.4723545222134844


In [39]:
print(weighted_loss(ax, ax))
print(weighted_loss(ay, ay))

-1.0
-0.5390491483180588


In [0]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    # +1 added to avoid 0/0 division
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)
  
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)
  
  
def weighted_loss(y_true, y_pred):
  loss1 = dice_coef(y_true, y_pred)
  loss2 = jacard_coef(y_true, y_pred)
  return -(loss1*0.4 + loss2*0.6)

In [0]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout2D, Activation
from keras import backend as K
from keras.layers.merge import concatenate
from keras.utils.data_utils import get_file

# Number of image channels (for example 3 in case of RGB, or 1 for grayscale images)
INPUT_CHANNELS = 3
# Number of output masks (1 in case you predict only one type of objects)
OUTPUT_MASK_CHANNELS = 1
# Pretrained weights




def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def double_conv_layer(x, size, dropout=0.10, batch_norm=True):
    if K.image_dim_ordering() == 'th':
        axis = 1
    else:
        axis = 3
    conv = Conv2D(size, (3, 3), padding='same')(x)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(size, (3, 3), padding='same')(conv)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    if dropout > 0:
        conv = SpatialDropout2D(dropout)(conv)
    return conv


def mini_UNET(dropout_val=0.40, weights=None): # No dropout by default
    if K.image_dim_ordering() == 'th':
        inputs = Input((INPUT_CHANNELS, 224, 224))
        axis = 1
    else:
        inputs = Input((224, 224, INPUT_CHANNELS))
        axis = 3
    filters = 8

    conv_224 = double_conv_layer(inputs, filters)
    pool_112 = MaxPooling2D(pool_size=(2, 2))(conv_224)

    conv_112 = double_conv_layer(pool_112, 2*filters)
    pool_56 = MaxPooling2D(pool_size=(2, 2))(conv_112)

    conv_56 = double_conv_layer(pool_56, 4*filters)
    pool_28 = MaxPooling2D(pool_size=(2, 2))(conv_56)

    conv_28 = double_conv_layer(pool_28, 8*filters)
    pool_14 = MaxPooling2D(pool_size=(2, 2))(conv_28)

    conv_14 = double_conv_layer(pool_14, 16*filters)
    pool_7 = MaxPooling2D(pool_size=(2, 2))(conv_14)

    conv_7 = double_conv_layer(pool_7, 32*filters)

    up_14 = concatenate([UpSampling2D(size=(2, 2))(conv_7), conv_14], axis=axis)
    up_conv_14 = double_conv_layer(up_14, 16*filters)

    up_28 = concatenate([UpSampling2D(size=(2, 2))(up_conv_14), conv_28], axis=axis)
    up_conv_28 = double_conv_layer(up_28, 8*filters)

    up_56 = concatenate([UpSampling2D(size=(2, 2))(up_conv_28), conv_56], axis=axis)
    up_conv_56 = double_conv_layer(up_56, 4*filters)

    up_112 = concatenate([UpSampling2D(size=(2, 2))(up_conv_56), conv_112], axis=axis)
    up_conv_112 = double_conv_layer(up_112, 2*filters)

    up_224 = concatenate([UpSampling2D(size=(2, 2))(up_conv_112), conv_224], axis=axis)
    up_conv_224 = double_conv_layer(up_224, filters, dropout_val)

    conv_final = Conv2D(OUTPUT_MASK_CHANNELS, (1, 1))(up_conv_224)
    conv_final = Activation('sigmoid', name='final_seg')(conv_final)
    
    
    # classification
    flat = Flatten()(conv_7)
    dense1 = Dense(512, activation='relu')(flat)
    dense2 = Dense(2)(dense1)
    class_layer = Activation('softmax', name='final_class')(dense2)
    model = Model(inputs, [conv_final, class_layer], name="mini_UNET")


    return model

In [55]:
model = mini_UNET()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_70 (Conv2D)              (None, 224, 224, 8)  224         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_67 (BatchNo (None, 224, 224, 8)  32          conv2d_70[0][0]                  
__________________________________________________________________________________________________
activation_71 (Activation)      (None, 224, 224, 8)  0           batch_normalization_67[0][0]     
__________________________________________________________________________________________________
conv2d_71 

In [0]:
from keras.models import load_model
from keras.optimizers import Adam
import os
from keras.losses import binary_crossentropy

out_model_path = 'unet_224_final_model.h5'
epochs = 50
patience = 20
batch_size = 64
optim_type = 'Adam'
learning_rate = 0.01

optim = Adam(lr=learning_rate)
loss={'final_seg': weighted_loss, 'final_class': 'binary_crossentropy'}
model.compile(optimizer=optim, loss=loss, metrics=[dice_coef, 'acc'])

In [60]:
y.shape

(138, 2)

In [0]:
history = model.fit(
    x = imgs,
    y = [masks,y],
    batch_size = 64,
    epochs=50,
    validation_split = 0.1,
    verbose=1)

Train on 124 samples, validate on 14 samples
Epoch 1/50
124/124 [==============================] - 70s 562ms/step - loss: 3.9965 - final_seg_loss: -1.4119 - final_class_loss: 5.4084 - final_seg_dice_coef: 1.2093 - final_seg_acc: 0.1711 - final_class_dice_coef: 0.4856 - final_class_acc: 0.5000 - val_loss: 8.5702e-08 - val_final_seg_loss: -2.3902e-08 - val_final_class_loss: 1.0960e-07 - val_final_seg_dice_coef: 2.3902e-08 - val_final_seg_acc: 0.7575 - val_final_class_dice_coef: 1.0000 - val_final_class_acc: 1.0000
Epoch 2/50
124/124 [==============================] - 60s 480ms/step - loss: 7.5920 - final_seg_loss: -2.7501 - final_class_loss: 10.3421 - final_seg_dice_coef: 1.5528 - final_seg_acc: 0.1945 - final_class_dice_coef: 0.3600 - final_class_acc: 0.3548 - val_loss: 8.5704e-08 - val_final_seg_loss: -2.3901e-08 - val_final_class_loss: 1.0960e-07 - val_final_seg_dice_coef: 2.3901e-08 - val_final_seg_acc: 0.7529 - val_final_class_dice_coef: 1.0000 - val_final_class_acc: 1.0000
Epoch 3/